In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import List

In [2]:
def compute_storage(train_dataset_sizes: List[int], model_size: int, prediction_x_size: int, prediction_y_size: int = 1,
                    KZG_BYTES=48, PED_BYTES=32, ECDSA_SIG_BYTES=64, ECDSA_PUBLICKEY_SIZE=32, SHA256_BYTES=32):

    n_elements_per_sample = prediction_x_size + prediction_y_size
    
    commitments = {
        "pc": {
            "commitment_train": KZG_BYTES * len(train_dataset_sizes),
            "commitment_model": KZG_BYTES,
            "commitment_prediction_x": KZG_BYTES, # we need to keep randomness but then we can not store the commitment, so essentially the same
            "commitment_prediction_y": KZG_BYTES,
            "trusted_setup": max(prediction_x_size, prediction_y_size) * KZG_BYTES
        },
        "pc_ipa": {
            "commitment_train": PED_BYTES * len(train_dataset_sizes),
            "commitment_model": PED_BYTES,
            "commitment_prediction_x": PED_BYTES,
            "commitment_prediction_y": PED_BYTES,
            "trusted_setup": 0,
        },
        "cerebro": {
            "commitment_train": sum([PED_BYTES * n_elements_per_sample * size for size in train_dataset_sizes]),
            "commitment_model": PED_BYTES * model_size,
            "commitment_prediction_x": PED_BYTES * prediction_x_size, # we could technically recompute these?
            "commitment_prediction_y": PED_BYTES * prediction_y_size,
            "trusted_setup": 0,
        },
        "sha3": {
            "commitment_train": SHA256_BYTES * len(train_dataset_sizes),
            "commitment_model": SHA256_BYTES,
            "commitment_prediction_x": SHA256_BYTES,
            "commitment_prediction_y": SHA256_BYTES,
            "trusted_setup": 0,
        }
    }

    common = {
        "signature_train": ECDSA_SIG_BYTES * len(train_dataset_sizes),
        "signature_modelowner": ECDSA_SIG_BYTES,
        "identity_train": ECDSA_PUBLICKEY_SIZE * len(train_dataset_sizes),
        "identity_modelowner": ECDSA_PUBLICKEY_SIZE
    }
    
    # to each entry in commitments add common items
    for commitment in commitments.values():
        commitment.update(common)
        
    return commitments



In [3]:
s_adult = compute_storage([8684, 8746, 8618], 2977, 91, 1)
s_mnist = compute_storage([20012, 20057, 19931], 431080, 784, 1)
s_cifar = compute_storage([16709, 16682, 16609], 3882058, 3072, 1)

# add to dataframe
df = pd.DataFrame([s_adult, s_mnist, s_cifar], index=["adult", "mnist", "cifar"])
df_unstacked = df.unstack().apply(pd.Series).reset_index()
df_unstacked = df_unstacked.rename(columns= {
    "level_0": "approach",
    "level_1": "dataset"
})

df_unstacked["storage_train"] = df_unstacked[["commitment_train", "signature_train", "identity_train"]].sum(axis=0)
df_unstacked["storage_inference"] = df_unstacked[["commitment_train", "signature_train", "identity_train", "commitment_model", "commitment_prediction_x", "commitment_prediction_y", "identity_modelowner"]].sum(axis=0)

display(df_unstacked)

df_unstacked.to_csv("storage.csv")

,approach,dataset,commitment_train,commitment_model,commitment_prediction_x,commitment_prediction_y,trusted_setup,signature_train,signature_modelowner,identity_train,identity_modelowner
0,pc_kzg,adult,144,48,48,48,4368,192,64,96,32
1,pc_kzg,mnist,144,48,48,48,37632,192,64,96,32
2,pc_kzg,cifar,144,48,48,48,147456,192,64,96,32
3,pc_ipa,adult,96,32,32,32,0,192,64,96,32
4,pc_ipa,mnist,96,32,32,32,0,192,64,96,32
5,pc_ipa,cifar,96,32,32,32,0,192,64,96,32
6,cerebro,adult,76685312,95264,2912,32,0,192,64,96,32
7,cerebro,mnist,1507200000,13794560,25088,32,0,192,64,96,32
8,cerebro,cifar,4916800000,124225856,98304,32,0,192,64,96,32
9,sha3,adult,96,32,32,32,0,192,64,96,32
